In [1]:
import sys
sys.path.append('/home/cryptobrahman/Own/football_competitions_research/own_modules/') 
sys.path.append('/mnt/KINGSTON_120/Own/football_competitions_research/own_modules/') 

In [2]:
import pandas as pd
import numpy as np
from numpy import arange, isclose

from flatlib import const
from flatlib import aspects
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.object import GenericObject
from flatlib.tools.chartdynamics import ChartDynamics
from flatlib.tools import arabicparts
from flatlib.dignities import essential

from _astro_constants import AstrologicalConstants, AstrologicalPoints, TransformDoubleValues, AspectsCalculate

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['14:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
df = pd.DataFrame(data)
df

,date,utc_time,lon,lat
0,2015/03/13,14:00,38n32,8w54
1,2016/05/23,12:30,08n07,18e14
2,2018/10/03,00:15,14s42,36w04


### Get astrological objects

In [5]:
df['charts'] = AstrologicalPoints.charts_calculate(df, 'date', 'utc_time', 'lon', 'lat')

In [6]:
df.charts

0    <flatlib.chart.Chart object at 0x7f433ae54250>
1    <flatlib.chart.Chart object at 0x7f433ae54280>
2    <flatlib.chart.Chart object at 0x7f433ae542b0>
Name: charts, dtype: object

In [7]:
necessary_constants = AstrologicalConstants.necessary_constants
str(necessary_constants)

"['SUN', 'MOON', 'SATURN', 'URANUS', 'NEPTUNE', 'PLUTO', 'CHIRON', 'NORTH_NODE', 'SOUTH_NODE']"

In [8]:
rulers_constants = AstrologicalConstants.rulers_constants
str(rulers_constants)

"['ASC', 'DESC', 'MC', 'IC', 'PARS_FORTUNA']"

In [9]:
antes_objects = AstrologicalConstants.for_antes_objects
str(antes_objects)

"['moon', 'ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'pars_fortuna', 'uranus', 'neptune', 'pluto', 'chiron']"

In [10]:
pars_constants = AstrologicalConstants.pars_constants
str(pars_constants)

"['PARS_SPIRIT', 'PARS_GLORY', 'PARS_CREST', 'PARS_ROCK']"

In [11]:
rulers_constant_col_names = AstrologicalPoints.rulers_col_names(rulers_constants)
rulers_constant_col_names

['ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna']

In [12]:
rulers_pars_col_names = AstrologicalPoints.rulers_col_names(pars_constants)
rulers_pars_col_names.append('ruler_antes_pars_fortuna')
str(rulers_pars_col_names)

"['ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock', 'ruler_antes_pars_fortuna']"

In [13]:
for obj in rulers_constants + necessary_constants + pars_constants:
    df[str.lower(obj)] = AstrologicalPoints.calculate_astro_objects(df.charts, obj)

In [14]:
for name, obj in zip(rulers_constant_col_names, rulers_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

In [15]:
for obj in antes_objects:
    antes_name = 'antes_' + obj
    df[antes_name] = AstrologicalPoints.antes_objects(df, obj)

In [16]:
antes_objects = []

for col in df.columns:
    if col.startswith('antes'):
        AstrologicalPoints.rename_antes_objects(df, col)
        antes_objects.append(col)      

In [17]:
pars_constants.append('ANTES_PARS_FORTUNA')

In [18]:
for name, obj in zip(rulers_pars_col_names, pars_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

In [19]:
houses_objects = AstrologicalConstants.houses_objects
houses_objects

['asc', 'desc', 'mc', 'ic']

In [20]:
str(antes_objects)

"['antes_moon', 'antes_ruler_asc', 'antes_ruler_desc', 'antes_ruler_mc', 'antes_ruler_ic', 'antes_pars_fortuna', 'antes_uranus', 'antes_neptune', 'antes_pluto', 'antes_chiron']"

In [21]:
pars_objects = AstrologicalConstants.pars_objects
str(pars_objects)

"['pars_fortuna', 'pars_spirit', 'pars_glory', 'pars_crest', 'pars_rock']"

In [22]:
ruler_objects = AstrologicalConstants.ruler_objects
str(ruler_objects)

"['ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna', 'ruler_antes_pars_fortuna', 'ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock']"

In [23]:
main_objects = AstrologicalConstants.main_objects
str(main_objects)

"['moon', 'sun', 'saturn', 'uranus', 'neptune', 'pluto', 'chiron', 'north_node', 'south_node']"

In [24]:
cols_for_id = houses_objects + antes_objects + pars_objects + ruler_objects + main_objects 
print(len(cols_for_id))
str(cols_for_id)

38


"['asc', 'desc', 'mc', 'ic', 'antes_moon', 'antes_ruler_asc', 'antes_ruler_desc', 'antes_ruler_mc', 'antes_ruler_ic', 'antes_pars_fortuna', 'antes_uranus', 'antes_neptune', 'antes_pluto', 'antes_chiron', 'pars_fortuna', 'pars_spirit', 'pars_glory', 'pars_crest', 'pars_rock', 'ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna', 'ruler_antes_pars_fortuna', 'ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock', 'moon', 'sun', 'saturn', 'uranus', 'neptune', 'pluto', 'chiron', 'north_node', 'south_node']"

In [25]:
df['id_for_aspects'] = AstrologicalPoints.id_for_aspekts(df, cols_for_id)

In [26]:
df['transform_id'] = df.id_for_aspects.map(lambda x: TransformDoubleValues(x).conbine_class_methods())

In [27]:
df.drop(columns='id_for_aspects', inplace=True)

In [28]:
df

,date,utc_time,lon,lat,charts,asc,desc,mc,ic,pars_fortuna,sun,moon,saturn,uranus,neptune,pluto,chiron,north_node,south_node,pars_spirit,pars_glory,pars_crest,pars_rock,ruler_asc,ruler_desc,ruler_mc,ruler_ic,ruler_pars_fortuna,antes_moon,antes_ruler_asc,antes_ruler_desc,antes_ruler_mc,antes_ruler_ic,antes_pars_fortuna,antes_uranus,antes_neptune,antes_pluto,antes_chiron,ruler_pars_spirit,ruler_pars_glory,ruler_pars_crest,ruler_pars_rock,ruler_antes_pars_fortuna,transform_id
0,2015/03/13,14:00,38n32,8w54,<flatlib.chart.Chart object at 0x7f433ae54250>,<Asc Cancer +27:24:38>,<Desc Capricorn +27:24:38>,<MC Aries +13:01:44>,<IC Libra +13:01:44>,<Pars Fortuna Aries +25:28:18 +00:00:00>,<Sun Pisces +22:39:56 +00:59:52>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Uranus Aries +15:06:33 +00:03:14>,<Neptune Pisces +07:50:38 +00:02:13>,<Pluto Capricorn +15:14:35 +00:01:02>,<Chiron Pisces +17:36:15 +00:03:43>,<North Node Libra +11:08:52 -00:03:11>,<South Node Aries +11:08:52 -00:03:11>,<Pars Spirit Libra +29:20:59 +00:00:00>,<Pars Glory Sagittarius +18:23:55 +00:00:00>,<Pars Crest Sagittarius +08:56:02 +00:00:00>,<Pars Rock Aries +09:40:27 +00:00:00>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Mars Aries +16:27:08 +00:45:19>,<Venus Aries +25:21:06 +01:12:41>,<Mars Aries +16:27:08 +00:45:19>,<Antes Moon Capricorn +09:16:25 +13:13:02>,<Antes Moon Capricorn +09:16:25 +13:13:02>,<Antes Saturn Capricorn +25:04:15 +00:00:06>,<Antes Mars Virgo +13:32:52 +00:45:19>,<Antes Venus Virgo +04:38:54 +01:12:41>,<Antes Pars Fortuna Virgo +04:31:42 +00:00:00>,<Antes Uranus Virgo +14:53:27 +00:03:14>,<Antes Neptune Libra +22:09:22 +00:02:13>,<Antes Pluto Sagittarius +14:45:25 +00:01:02>,<Antes Chiron Libra +12:23:45 +00:03:43>,<Venus Aries +25:21:06 +01:12:41>,<Jupiter Leo +13:39:12 -00:04:47>,<Jupiter Leo +13:39:12 -00:04:47>,<Mars Aries +16:27:08 +00:45:19>,<Mercury Pisces +00:37:44 +01:29:39>,"[(asc, Asc), (desc, Desc), (mc, MC), (ic, IC), (antes_ruler_desc, Antes Saturn), (antes_ruler_mc..."
1,2016/05/23,12:30,08n07,18e14,<flatlib.chart.Chart object at 0x7f433ae54280>,<Asc Virgo +27:12:45>,<Desc Pisces +27:12:45>,<MC Gemini +27:30:28>,<IC Sagittarius +27:30:28>,<Pars Fortuna Aries +15:36:29 +00:00:00>,<Sun Gemini +02:47:58 +00:57:39>,<Moon Sagittarius +21:11:41 +12:19:41>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Uranus Aries +22:47:40 +00:02:49>,<Neptune Pisces +11:54:55 +00:00:42>,<Pluto Capricorn +17:11:35 -00:00:57>,<Chiron Pisces +24:43:00 +00:01:47>,<North Node Virgo +18:00:29 -00:03:11>,<South Node Pisces +18:00:29 -00:03:11>,<Pars Spirit Pisces +08:49:02 +00:00:00>,<Pars Glory Capricorn +07:57:50 +00:00:00>,<Pars Crest Virgo +14:43:46 +00:00:00>,<Pars Rock Aries +08:19:08 +00:00:00>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mars Sagittarius +01:25:22 -00:21:00>,<Antes Moon Capricorn +08:48:19 +12:19:41>,<Antes Mercury Leo +15:37:37 +00:04:22>,<Antes Jupiter Aries +16:26:57 +00:02:30>,<Antes Mercury Leo +15:37:37 +00:04:22>,<Antes Jupiter Aries +16:26:57 +00:02:30>,<Antes Pars Fortuna Virgo +14:23:31 +00:00:00>,<Antes Uranus Virgo +07:12:20 +00:02:49>,<Antes Neptune Libra +18:05:05 +00:00:42>,<Antes Pluto Sagittarius +12:48:25 -00:00:57>,<Antes Chiron Libra +05:17:00 +00:01:47>,<Jupiter Virgo +13:33:03 +00:02:30>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Mercury Taurus +14:22:23 +00:04:22>,<Mars Sagittarius +01:25:22 -00:21:00>,<Mercury Taurus +14:22:23 +00:04:22>,"[(asc, Asc), (desc, Desc), (mc, MC), (ic, IC), (antes_moon, Antes Moon), (antes_pars_fortuna, An..."
2,2018/10/03,00:15,14s42,36w04,<flatlib.chart.Chart object at 0x7f433ae542b0>,<Asc Gemini +05:25:08>,<Desc Sagittarius +05:25:08>,<MC Pisces +07:39:39>,<IC Virgo +07:39:39>,<Pars Fortuna Pisces +13:18:26 +00:00:00>,<Sun Libra +09:44:33 +00:59:03>,<Moon Cancer +17:37:52 +14:05:52>,<Saturn Capricorn +03:06:46 +00:

### Calculate aspects

In [29]:
# stop

In [30]:
from _astro_constants import GeneralMethods

In [31]:
objects = AspectsCalculate(df.iloc[0]).conbine_class_methods()

In [32]:
# objects

In [33]:
singles_degrees = AstrologicalConstants.singles_degrees
singles_degrees

[0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]

In [34]:
def remove_objects(objects: list, remove_objs: list):
    list_wt_objs = [x for x in objects if x.id not in remove_objs]
    return list_wt_objs

In [35]:
def orb_calculate(include_lon: float, obj_lon: float, deg: int):
    obj_lon_deg_plus = GeneralMethods.plus_minus_obj_lon_deg(obj_lon, deg, '+')
    obj_lon_deg_minus = GeneralMethods.plus_minus_obj_lon_deg(obj_lon, deg, '-')
    
    orb_plus = abs(include_lon - obj_lon_deg_plus)
    orb_minus = abs(include_lon - obj_lon_deg_minus)
    
    if obj_lon_deg_plus == obj_lon_deg_minus:
        orb_plus = 360 - orb_plus

    if orb_plus < orb_minus:
        return orb_plus
    else:
        return orb_minus

In [36]:
# def max_degree_calc(obj_lon: float, obj_orb: int, deg: int):
#     if obj_lon + deg + obj_orb >= 360:
#         max_degree = 360
#     elif obj_lon - deg - obj_orb < 0:
#         max_degree = 360
#     else:
#         max_degree = 0
#     return max_degree

In [37]:
def pos_neg_obj_lon_deg(obj_lon: float, deg: int):
    pos_obj_lon_deg = obj_lon + deg
    neg_obj_lon_deg = obj_lon - deg
    
    if pos_obj_lon_deg >= 360:
        pos_obj_lon_deg = abs(360 - pos_obj_lon_deg)
    if neg_obj_lon_deg < 0:
        neg_obj_lon_deg = abs(360 + neg_obj_lon_deg)  

    return pos_obj_lon_deg, neg_obj_lon_deg

In [144]:
def transform_list_type(list_blocks: list):
    all_list_blocks = []
    for block in list_blocks:
        if type(block[0]) is list:
            all_list_blocks.append(block[0]) 
            all_list_blocks.append(block[1]) 
        else:
            all_list_blocks.append(block)
    return all_list_blocks        

In [229]:
def remove_list_duble(list_blocks: list):
    for ind, block in enumerate(list_blocks):
        if list_blocks.count(block) > 1:
            list_blocks.pop(ind)
            remove_list_duble(list_blocks)

In [255]:
def pos_neg_points_orb(obj_lon: float, obj_orb: int, deg: int):
    pos_obj_lon_deg, neg_obj_lon_deg = pos_neg_obj_lon_deg(obj_lon, deg)
    print(pos_obj_lon_deg, neg_obj_lon_deg) 
    bef_pos_point_orb = pos_obj_lon_deg - obj_orb
    bef_neg_point_orb = neg_obj_lon_deg - obj_orb
    aft_pos_point_orb = pos_obj_lon_deg + obj_orb
    aft_neg_point_orb = neg_obj_lon_deg + obj_orb
    
    block_1 = [bef_pos_point_orb, pos_obj_lon_deg] 
    block_2 = [pos_obj_lon_deg,   aft_pos_point_orb] 
    block_3 = [bef_neg_point_orb, neg_obj_lon_deg]
    block_4 = [neg_obj_lon_deg,   aft_neg_point_orb]
    
    if bef_pos_point_orb < 0:
        block_1 = [[360 + bef_pos_point_orb, 359.99], [0, pos_obj_lon_deg]] 
    if aft_pos_point_orb >= 360:
        block_2 = [[pos_obj_lon_deg, 359.99], [0, aft_pos_point_orb - 360]]     
    if bef_neg_point_orb < 0:    
        block_3 = [[360 + bef_neg_point_orb, 359.99], [0, neg_obj_lon_deg]]
    if aft_neg_point_orb >= 360:
        block_4 = [[neg_obj_lon_deg, 359.99], [0, aft_neg_point_orb - 360]]  
    
    all_obj_lon_degs = [block_1, block_2, block_3, block_4]
    all_obj_lon_degs = transform_list_type(all_obj_lon_degs)
    all_obj_lon_degs = remove_list_duble(all_obj_lon_degs)
    
    return all_obj_lon_degs      

In [257]:
x = pos_neg_points_orb(obj_lon=355, obj_orb=20, deg=0)
x

355 355


In [41]:
# pos_neg_points_orb(5, 10, 0, before_aspect='yes')

In [ ]:
pos_neg_points_orb(5, 10, 0, before_aspect='no')

In [ ]:
def switching_ranges_with_orb(include_lon: float, obj_lon: float, obj_orb: int, deg: int, before_aspect = 'no'):
    if before_aspect != 'no' and before_aspect != 'yes':
        print("Unsuported value in before_aspect")
    
    max_degree = max_degree_calc(obj_lon, obj_orb, deg)
    if before_aspect == 'yes':
        obj_orb = -obj_orb    
    print(max_degree)
    pos_arr_start = obj_lon - max_degree + deg
    pos_arr_end = obj_lon + obj_orb - max_degree + deg
    neg_arr_start = max_degree + obj_lon - deg
    neg_arr_end = max_degree + obj_lon + obj_orb - deg
    
    if before_aspect == 'no' and (isclose(round(include_lon, 2), arange(pos_arr_start, pos_arr_end, 0.01)).any() or \
    isclose(round(include_lon, 2), arange(neg_arr_start, neg_arr_end, 0.01)).any()):
        orb_con = orb_calculate(include_lon, obj_lon, deg)
    elif before_aspect == 'yes' and (isclose(round(include_lon, 2), arange(pos_arr_end, pos_arr_start, 0.01)).any() or \
    isclose(round(include_lon, 2), arange(neg_arr_end, neg_arr_start, 0.01)).any()):
        orb_con = orb_calculate(include_lon, obj_lon, deg)
    else:
        orb_con = 'None'
    return orb_con

In [ ]:
# orb_con = switching_ranges_with_orb(22.10, 12.0, 13, 0, before_aspect='no')
# orb_con

In [ ]:
orb_con = switching_ranges_with_orb(350.10, 1.10, 13, 0, before_aspect='yes')
orb_con

In [ ]:
def object_data(objects: list, name: str):
    for obj in objects:
        if obj.id == name:
            if 'meanMotion' not in dir(obj):
                obj_house = 1
                obj_lonsp = 0
            else:
                obj_house = 0
                obj_lonsp = obj.lonspeed
            
            obj_id, obj_lon, obj_orb = obj.id, obj.lon, obj.orb()
            return obj_id, obj_lon, obj_orb, obj_lonsp, obj_house

In [ ]:
def house_type_approuch(incl_lon: float, incl_lonsp: float, obj_lon: float):
    if obj_lon <= incl_lon and round(obj_lon/100) == round(incl_lon/100):
        pos = 'in'
    elif obj_lon >= incl_lon and round(obj_lon/100) == round(incl_lon/100):
        pos = 'out'
    else:
        if  obj_lon >= incl_lon:
            pos = 'in'
        else:
            pos = 'out'

    if incl_lonsp == 0:
        type_appr = pos + '_stat'
    elif (pos == 'in' and incl_lonsp < 0) or (pos == 'out' and incl_lonsp > 0):
        type_appr = pos + '_conv'
    else:
        type_appr = pos + '_diver'
    
    return type_appr

In [ ]:
def left_right_semi_circle(incl_lon: float, obj_lon: float, deg: int):
    obj_lon_deg = obj_lon + deg
    
    if obj_lon_deg > 360:
        obj_lon_deg = abs(360 - obj_lon_deg)
    
    if obj_lon_deg <= 180 and isclose(round(incl_lon, 2), arange(obj_lon_deg, obj_lon_deg + 180, 0.01)).any():
        incl_lon_pos = 'right'
    elif obj_lon_deg > 180 and (isclose(round(incl_lon, 2), arange(obj_lon_deg, 359.99, 0.01)).any() or \
    isclose(round(incl_lon, 2), arange(0, 360 - obj_lon_deg, 0.01)).any()):
        incl_lon_pos = 'right'
    else:
        incl_lon_pos = 'left'
    return incl_lon_pos    

In [ ]:
def type_approach(incl_lon: float, incl_lonsp: float, obj_lon: float, obj_lonsp: float, obj_house: int, deg: int):    
    incl_lon_pos = left_right_semi_circle(incl_lon, obj_lon, deg)
   
    if obj_house == 1:
        type_appr = house_type_approuch(incl_lon, incl_lonsp, obj_lon)
    else:
        if incl_lon_pos == 'left':
            l1_lonsp = incl_lonsp
            l2_lonsp = obj_lonsp 
        else:
            l1_lonsp = obj_lonsp 
            l2_lonsp = incl_lonsp 
        
        if incl_lonsp == obj_lonsp:
            type_appr = 'stat'
        
        elif (l1_lonsp == 0 and l2_lonsp > 0)  or \
             (l1_lonsp < 0  and l2_lonsp == 0) or \
             (l1_lonsp < 0  and l2_lonsp > 0)  or \
             (l2_lonsp > l1_lonsp > 0) or \
             (l1_lonsp < l2_lonsp < 0): # -0.2 speedly than -0.1
             type_appr = 'diver'
        elif (l1_lonsp == 0 and l2_lonsp < 0)  or \
             (l1_lonsp > 0  and l2_lonsp == 0) or \
             (l1_lonsp > 0  and l2_lonsp < 0)  or \
             (l1_lonsp > l2_lonsp > 0) or \
             (l2_lonsp < l1_lonsp < 0):
             type_appr = 'conv'
        else:
             type_appr = 'no'
    
    return type_appr    

In [ ]:
def type_orb_calculate(obj_orb: int, before_aspect: str, after_orb ='const', before_orb ='const'):
    if before_aspect == 'no' and after_orb != 'const' and before_orb == 'const':
        orb = after_orb
    elif before_aspect == 'yes' and before_orb != 'const' and after_orb == 'const':
        orb = before_orb 
    else:
        orb = obj_orb
    return orb

In [ ]:
def object_aspects(id_obj: str, unique_objs: list, remove_objs: list, aspekts_degrees: dict, before_aspect: str, after_orb ='const', before_orb ='const'):
    if (after_orb !='const' and type(after_orb) is not int) or (before_orb !='const' and type(before_orb) is not int):
        print("Unsuported value in conv_orb or diver_orb")
    
    singles_degrees = AstrologicalConstants.singles_degrees
    
    all_aspekts = []
    objects_cp = unique_objs.copy()
    
    obj_id, obj_lon, obj_orb, obj_lonsp, obj_house = object_data(objects_cp, id_obj)
    obj_orb = type_orb_calculate(obj_orb, before_aspect, after_orb, before_orb)    
    objects_cp = remove_objects(objects_cp, [id_obj] + remove_objs)
    
    for incl_obj in objects_cp:
        for type_asp, deg in aspekts_degrees.items():
            orb_con = switching_ranges_with_orb(incl_obj.lon, obj_lon, obj_orb, deg, before_aspect)
            
            if orb_con:
                asp = type_asp
                type_appr = type_approach(incl_obj.lon, incl_obj.lonspeed, obj_lon, obj_lonsp, obj_house, deg)
                orb_con = GeneralMethods.degree_transform(orb_con)
                sing = GeneralMethods.equal_different_sing_feature(singles_degrees, incl_obj.lon, obj_lon, deg)
            else:
                asp, type_appr, orb_con, sing = None, None, None, None
              
            point_aspects = {'f_point': id_obj, 's_point': incl_obj.id, 'type': asp, 'approach': type_appr, 'sing': sing, 'orb': orb_con}

            if point_aspects['type'] != None:
                all_aspekts.append(point_aspects)

    return all_aspekts

In [ ]:
houses_remove_objs = []
moon_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Uranus', 'Neptune', 'Pluto', 'Chiron', 'North Node', 'South Node', 'Antes Moon','Antes Uranus', 
                      'Antes Neptune', 'Antes Pluto', 'Antes Chiron']
main_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Moon', 'Antes Moon']

In [ ]:
houses_aspect_degs = {'Con': 0}
moon_aspect_degs = AstrologicalConstants.aspekts_degrees
main_aspect_degs = {'Con': 0, 'Opp': 180}

In [ ]:
house_objs = ['Asc', 'Desc', 'MC', 'IC'] 

In [ ]:
# moon_asp_after = object_aspects('Moon', objects, moon_remove_objs, moon_aspect_degs, before_aspect='no')
# moon_asp_before = object_aspects('Moon', objects, moon_remove_objs, moon_aspect_degs, before_aspect='yes', before_orb=1)
# all_moon_asp = moon_asp_after + moon_asp_before

In [ ]:
# all_house_asp = []
# for h_obj in house_objs:
#     house_asp_after = object_aspects(h_obj, objects, houses_remove_objs, houses_aspect_degs, before_aspect='no', after_orb=3)
#     house_asp_before = object_aspects(h_obj, objects, houses_remove_objs, houses_aspect_degs, before_aspect='yes', before_orb=3)
#     all_house_asp.append(house_asp_after)
#     all_house_asp.append(house_asp_before)

In [ ]:
# objects = AspectsCalculate(df.iloc[0]).conbine_class_methods()

In [ ]:
for x in objects:
    if x.id == 'Asc':
        x.lon = 2.1
    if x.id == 'Pluto':
        x.lon = 1.1
    if x.id == 'Neptune':
        x.lon = 3.1 

In [ ]:
asc_pluto = []
for obj in objects:
    if obj.id == 'Asc' or obj.id == 'Pluto' or obj.id == 'Neptune':
        asc_pluto.append(obj)
asc_pluto        

In [ ]:
# asc_pluto_asp_after = object_aspects('Asc', objects, houses_remove_objs, houses_aspect_degs, before_aspect='no', after_orb=90)
# asc_pluto_asp_after

In [ ]:
str([x.lon for x in objects])

In [ ]:
# all_main_asp = []
# for m_obj in objects:
#     main_asp_after = object_aspects(m_obj.id, objects, main_remove_objs, main_aspect_degs, before_aspect='no', after_orb=3)
#     main_asp_before = object_aspects(m_obj.id, objects, main_remove_objs, main_aspect_degs, before_aspect='yes', before_orb=3)
#     all_main_asp.append(main_asp_after)
#     all_main_asp.append(main_asp_before)

In [ ]:
all_aspects = all_moon_asp + all_house_asp + all_main_asp + asc_pluto_asp_after
all_aspects_full = []
for asp in all_aspects:
    if asp != []:
        all_aspects_full.append(asp)

In [ ]:
# all_aspects_full

In [ ]:
[x.lon for x in objects if x.id == 'IC']

In [ ]:
[x.lon for x in objects if x.id == 'Antes Neptune']

In [ ]:
# [x.lonspeed for x in objects if x.id == 'Moon']

In [ ]:
[x.lon for x in objects if x.id == 'Pars Spirit']

In [ ]:
# [x.lonspeed for x in objects if x.id == 'Mercury']

In [ ]:
[x.lon for x in objects if x.id == 'Antes Pars Fortuna']

In [ ]:
str([x.id for x in objects])

In [ ]:
# for x in objects:
#     if x.id == 'Asc':
#         x.lon = 1
#     if x.id == 'Pluto':
#         x.lon = 15
#     if x.id == 'MC':
#         x.lon = 303      
    # if x.id == 'Neptune':
    #     x.lon = 4
#     if x.id == 'Saturn':
#         x.lon = 34.28
#     if x.id == 'Venus':
#         x.lon = 37.01  

In [ ]:
str(['conv_compl', 'conv_denide']) # for septener planet and parses

In [ ]:
# Movement of planets for conv or diver aspect type
# Rules for aspekts:
# Remove: 
# Antes Moon - only house conuction
# Doublecation similar planets and antes aspects  
# Houses opposition
# Add:
# Will exist aspect function for rulers Asc and Dsc
# Divergent or convergent 
# Pluto and Asc conuction if Pluto near Asc with 30 degree orb
# Denide of aspect from aspect with other planet (create after getting all point aspects)

In [ ]:
chart = df.charts.values[0]
chart

In [ ]:
asc = chart.get(const.ASC)
moon = chart.get(const.MOON)
sun = chart.get(const.SUN)

In [ ]:
from flatlib.object import Object

In [ ]:
sun = chart.get(const.SUN)

In [ ]:
# antes_pars_fortuna_obj 

In [ ]:
sun.lon

In [ ]:
asc_obj = Object.fromDict({'id': asc.id, 'sign': asc.sign, 'signlon': asc.signlon, 'lonspeed': 0, 'orb': asc.orb})

In [ ]:
moon_obj = Object.fromDict({'id': moon.id, 'sign': moon.sign, 'signlon': moon.signlon, 'lonspeed': moon.lonspeed, 'orb': moon.orb})

In [ ]:
antes_pars_fortuna_obj = Object.fromDict({'id': 'Antes Pars Fortuna'})#, 'sign': sun.sign, 'signlon': sun.signlon, 'lonspeed': sun.lonspeed})#, 'orb': sun.orb})

In [ ]:
antes_pars_fortuna_obj.orb()

In [ ]:
moon_asc_aspect = aspects.getAspect(moon_obj, asc_obj, const.MAJOR_ASPECTS)
print('moon_asc_aspect: ', moon_asc_aspect)

In [ ]:
dyn = ChartDynamics(chart)

In [ ]:
moon_asps = dyn.immediateAspects('North Node', const.MAIN_ASPECTS)
print(moon_asps)  

In [ ]:
from flatlib.lists import GenericList

In [ ]:
list_obj = [antes_pars_fortuna_obj]

In [ ]:
gen_list = GenericList(list_obj)
gen_list

In [ ]:
gen_list.add = sun_obj

In [ ]:
gen_list.content

In [ ]:
chart.get('Antes Pars Fortuna')

In [ ]:
chart.objects.add(antes_pars_fortuna_obj)

In [ ]:
chr_obj = [x for x in chart.objects]

In [ ]:
len (chr_obj)

In [ ]:
chr_obj[15].id

In [ ]:
dyn_obj = ChartDynamics(chart)

In [ ]:
moon_asps = dyn_obj.immediateAspects('Moon', const.MAJOR_ASPECTS)
moon_asps

In [ ]:
chart.getObject(ID='Moon')

In [ ]:
df.antes_moon[0]

In [ ]:
# Create new columns for each object, five clasess for favorite's advantsge and orbs's 